In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../Data/processed/processed_data.csv')

print(f"Valeurs manquantes : {df['clean_text'].isnull().sum()}")

Valeurs manquantes : 10


In [4]:
df = df.dropna(subset=['clean_text'])
print(f"Valeurs manquantes : {df['clean_text'].isnull().sum()}")

Valeurs manquantes : 0


In [5]:
print(f"Taille du dataset chargé : {df.shape}")
display(df.head())

Taille du dataset chargé : (28801, 3)


,label,label_text,clean_text
0,1,spam,softwar understand oem softwar lead temptat fi...
1,0,ham,perspect ferc regulatori action client conf ca...
2,1,spam,want tri ci li thought way expens viagra per d...
3,0,ham,enron hpl actual decemb teco tap enron hpl ga ...
4,1,spam,look cheap high qualiti softwar rotat napoleon...


###  Séparation du dataset en données d'entraînement et de test

Dans cette étape, nous séparons notre dataset en deux parties :

- **80%** des données pour l'entraînement du modèle (`X_train`, `y_train`)
- **20%** des données pour le test et l'évaluation (`X_test`, `y_test`)

Cette séparation permet de mesurer correctement les performances du modèle sur des données qu’il n’a jamais vues.  
Le paramètre `random_state=42` assure la reproductibilité : chaque exécution donnera le même split.

Nous travaillons ici avec :
- **X** : la colonne `clean_text` (les textes prétraités)
- **y** : la colonne `label` (la classe associée à chaque texte)


In [6]:
from sklearn.model_selection import train_test_split

# Sélection des features (X) et de la variable cible (y)
X = df['clean_text']     # Textes nettoyés
y = df['label']          # Labels : 0 = ham, 1 = spam 

# Division du dataset : 80% entraînement, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,        # 20% des données utilisées pour le test
    random_state=42       # Assure que le split reste identique à chaque exécution
)

# Affichage des tailles des deux ensembles
print(f"Training emails: {X_train.shape[0]}")
print(f"Testing emails : {X_test.shape[0]}")


Training emails: 23040
Testing emails : 5761


###  Vectorisation avec TF-IDF et sauvegarde du vectorizer

Dans cette cellule nous transformons les textes en vecteurs numériques avec **TfidfVectorizer** et sauvegardons le vectorizer pour réutilisation future.

**Étapes réalisées :**
1. Création d'un dossier `../models` si nécessaire pour y stocker les artefacts (vectorizer, modèles, etc.).  
2. Instanciation du `TfidfVectorizer` avec un maximum de **5000** features (`max_features=5000`) pour limiter la dimensionnalité.  
3. **Fit** sur l'ensemble d'entraînement (`X_train`) pour apprendre le vocabulaire et transformer les textes en vecteurs TF-IDF.  
4. **Transformation** des textes de test (`X_test`) en vecteurs TF-IDF en utilisant le vocabulaire appris.  
5. Sauvegarde du vectorizer sur disque (`pickle`) dans `../saved_models/tfidf_vectorizer.pkl`.  

**Pourquoi fit sur X_train seulement ?**  
Pour éviter la fuite de données : le modèle ne doit jamais "voir" les données de test lors de l'apprentissage du vocabulaire.

Les matrices `X_train_tfidf` et `X_test_tfidf` sont prêtes pour l’entraînement d’un modèle de classification.


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import os

# --- Créer le dossier 'saved_models' s'il n'existe pas ---
os.makedirs('../models', exist_ok=True)

# --- Initialisation du vectorizer ---
tfidf = TfidfVectorizer(max_features=5000)  # limite à 5000 tokens les plus informatifs

# --- Fit sur l'entraînement et transformation ---
X_train_tfidf = tfidf.fit_transform(X_train)  # apprentissage du vocabulaire + transformation
X_test_tfidf = tfidf.transform(X_test)        # transformation du test avec le vocabulaire appris

# --- Sauvegarde du vectorizer pour réutilisation ---
with open('../models/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# --- Informations pour vérification ---
print("Vectorization complete.")
print(f"Train Shape: {X_train_tfidf.shape}")  # exemple: (23048, 5000)
print(f"Test Shape : {X_test_tfidf.shape}")   # exemple: (5763, 5000)
print("Vectorizer saved to '../models/tfidf_vectorizer.pkl'")



Vectorization complete.
Train Shape: (23040, 5000)
Test Shape : (5761, 5000)
Vectorizer saved to '../models/tfidf_vectorizer.pkl'


###  Sauvegarde des matrices TF-IDF et des labels

Dans cette étape, nous sauvegardons les données transformées afin de pouvoir les réutiliser ultérieurement sans avoir à recalculer les vecteurs TF-IDF.  

**Détails :**

1. **Matrices TF-IDF (`X_train_tfidf` et `X_test_tfidf`)**  
   - Ces matrices sont **sparse** (beaucoup de zéros) car chaque texte n'utilise qu'une partie du vocabulaire.  
   - Nous les sauvegardons au format **`.npz`** grâce à `scipy.sparse.save_npz` :  
     - Ce format est très efficace en espace mémoire.  
     - Permet de recharger rapidement les matrices plus tard pour l’entraînement ou l’inférence.

2. **Labels (`y_train` et `y_test`)**  
   - Sauvegardés au format **CSV** pour faciliter la lecture et l’utilisation avec pandas.  

**Structure des fichiers sauvegardés :**
- `../data/processed/X_train_tfidf.npz` → matrice TF-IDF d'entraînement  
- `../data/processed/X_test_tfidf.npz` → matrice TF-IDF de test  
- `../data/processed/y_train.csv` → labels d'entraînement  
- `../data/processed/y_test.csv` → labels de test  

> Ces fichiers constituent un pipeline complet de données prétraitées et vectorisées, prêts pour l’entraînement d’un modèle de classification.


In [9]:
import scipy.sparse
import os

# --- Créer le dossier 'processed' s'il n'existe pas ---
os.makedirs('../data/processed', exist_ok=True)

# --- Sauvegarde des matrices TF-IDF ---
# Les matrices TF-IDF sont sparse (beaucoup de zéros)
# Le format .npz est optimisé pour les matrices clairsemées et économise de l'espace disque
scipy.sparse.save_npz('../data/processed/X_train_tfidf.npz', X_train_tfidf)
scipy.sparse.save_npz('../data/processed/X_test_tfidf.npz', X_test_tfidf)

# --- Sauvegarde des labels correspondants ---
# Les labels sont sauvegardés au format CSV pour une lecture facile avec pandas
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

# --- Informations de sortie ---
print("Matrices TF-IDF et labels sauvegardés avec succès !")
print("X_train_tfidf -> ../data/processed/X_train_tfidf.npz")
print("X_test_tfidf  -> ../data/processed/X_test_tfidf.npz")
print("y_train       -> ../data/processed/y_train.csv")
print("y_test        -> ../data/processed/y_test.csv")


Matrices TF-IDF et labels sauvegardés avec succès !
X_train_tfidf -> ../data/processed/X_train_tfidf.npz
X_test_tfidf  -> ../data/processed/X_test_tfidf.npz
y_train       -> ../data/processed/y_train.csv
y_test        -> ../data/processed/y_test.csv
